In [2]:
from pyspark.sql import SparkSession
import getpass 
username = getpass.getuser()
spark = SparkSession.\
    builder.\
    config('spark.ui.port','0').\
    config("spark.sql.warehouse.dir",f"/user/itv010130/warehouse").\
    enableHiveSupport().\
    master('yarn').\
    getOrCreate()

In [41]:
products_df = spark.read \
.format("csv") \
.option("header","true") \
.option("inferSchema","true") \
.load("/public/trendytech/retail_db/products")

In [4]:
products_df.show()

+---+---+---------------------------------------------+----+------+-------------------------------------------------------------------------------------+
|  1|  2|Quest Q64 10 FT. x 10 FT. Slant Leg Instant U| _c3| 59.98|http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy|
+---+---+---------------------------------------------+----+------+-------------------------------------------------------------------------------------+
|  2|  2|                         Under Armour Men'...|null|129.99|                                                                 http://images.acm...|
|  3|  2|                         Under Armour Men'...|null| 89.99|                                                                 http://images.acm...|
|  4|  2|                         Under Armour Men'...|null| 89.99|                                                                 http://images.acm...|
|  5|  2|                         Riddell Youth Rev...|null|199.99|         

In [6]:
products_df.printSchema()

root
 |-- 1: integer (nullable = true)
 |-- 2: integer (nullable = true)
 |-- Quest Q64 10 FT. x 10 FT. Slant Leg Instant U: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- 59.98: double (nullable = true)
 |-- http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy: string (nullable = true)



In [9]:
products_df.createOrReplaceTempView("products")

In [11]:
spark.sql("create table itv010130_retail_db.productss_list (ProductID integer, Category integer, ProductName string, Description string, Price double, ImageURL string) using csv")

""


In [12]:
spark.sql("insert into itv010130_retail_db.productss_list select * from products")

""


In [13]:
result = spark.sql("select * from itv010130_retail_db.productss_list")

In [14]:
result.show(3)

+---------+--------+--------------------+-----------+------+--------------------+
|ProductID|Category|         ProductName|Description| Price|            ImageURL|
+---------+--------+--------------------+-----------+------+--------------------+
|        2|       2|Under Armour Men'...|       null|129.99|http://images.acm...|
|        3|       2|Under Armour Men'...|       null| 89.99|http://images.acm...|
|        4|       2|Under Armour Men'...|       null| 89.99|http://images.acm...|
+---------+--------+--------------------+-----------+------+--------------------+
only showing top 3 rows



### 1. Find the total number of products

In [23]:
result = spark.sql("select count(ProductName) as Total_products from itv010130_retail_db.productss_list")

In [24]:
result.show()

+--------------+
|Total_products|
+--------------+
|          1344|
+--------------+



### 2. Number of unique categories of products

In [28]:
result = spark.sql("select count(distinct Category) as Number_category from itv010130_retail_db.productss_list")

In [35]:
result.show()

+---------------+
|Number_category|
+---------------+
|             55|
+---------------+



### 3. Top 5 most expensive products

In [44]:
most_expensive = spark.sql("select ProductName, Category, Price, imageURL from itv010130_retail_db.productss_list order by price desc limit 5")

In [46]:
most_expensive.show(truncate = False)

+------------------------------------------------+--------+-------+-----------------------------------------------------------------------------------+
|ProductName                                     |Category|Price  |imageURL                                                                           |
+------------------------------------------------+--------+-------+-----------------------------------------------------------------------------------+
|SOLE E35 Elliptical                             |10      |1999.99|http://images.acmesports.sports/SOLE+E35+Elliptical                                |
|SOLE F85 Treadmill                              |4       |1799.99|http://images.acmesports.sports/SOLE+F85+Treadmill                                 |
|SOLE F85 Treadmill                              |10      |1799.99|http://images.acmesports.sports/SOLE+F85+Treadmill                                 |
|SOLE F85 Treadmill                              |22      |1799.99|http://images.acmespo

### 4. Number of products in each category with price greater than $100

In [52]:
cat_product100 = spark.sql("select Category,count(ProductName) as product_count from itv010130_retail_db.productss_list where Price > 100 group by Category")

In [53]:
cat_product100.show()

+--------+-------------+
|Category|product_count|
+--------+-------------+
|      31|           17|
|      53|           16|
|      34|           15|
|      44|            9|
|      12|            3|
|      22|            4|
|      47|           10|
|      52|            5|
|      13|            1|
|       6|            5|
|      16|           11|
|       3|            5|
|      20|            7|
|      57|            6|
|      54|            6|
|      48|           17|
|       5|           11|
|      19|           13|
|      41|           11|
|      43|           23|
+--------+-------------+
only showing top 20 rows



### 5. Names and Prices of Products greater than 200 dollar and in category 5

In [54]:
Product = spark.sql("select ProductName, Price from itv010130_retail_db.productss_list where Price > 200 and Category = 5")

In [56]:
Product.show(truncate = False)

+------------------------------------------------+------+
|ProductName                                     |Price |
+------------------------------------------------+------+
|"Goaliath 54"" In-Ground Basketball Hoop with P"|499.99|
|Fitness Gear 300 lb Olympic Weight Set          |209.99|
|Teeter Hang Ups NXT-S Inversion Table           |299.99|
+------------------------------------------------+------+

